In [186]:
import numpy as np
import pandas as pd
import scipy
from scipy.optimize import minimize
import plotly.express as px
import plotly.graph_objects as go
import datetime

In [187]:
import warnings
warnings.filterwarnings('ignore')

In [188]:
approval = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vS-FKWVTTFtJT6u56e0bqdfoMcXvDO1DUChsJ3jQAMB2lZk2SMqVfmg7dGjclTYkYWz-Pm5lfcLPjp4/pub?output=csv')
# approval = approval.reset_index()
# approval = approval.rename({'index':'poll_id'}, axis=1)
approval

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,tracking,approve,disapprove,net,adjusted_approve,adjusted_disapprove,adjusted_net,url,createddate,timestamp
0,Donald Trump (2nd term),Adults,Morning Consult,NaN,5/2/2025,5/8/2025,45000.0,A,1.718826,1.718826,NaN,46.24,48.04,-1.799999,44.89884,50.69795,-5.799107,https://pro.morningconsult.com/trackers/global...,5/8/2025,14:11:51 10 May 2025
1,Donald Trump (2nd term),All polls,Morning Consult,NaN,5/2/2025,5/8/2025,45000.0,A,1.236251,1.236251,NaN,46.24,48.04,-1.799999,45.31192,49.17940,-3.867481,https://pro.morningconsult.com/trackers/global...,5/8/2025,14:11:47 10 May 2025
2,Donald Trump (2nd term),Adults,TIPP Insights,NaN,4/30/2025,5/2/2025,1400.0,A,1.550809,1.197667,NaN,42.00,47.00,-5.000000,42.60161,49.48451,-6.882900,https://tippinsights.com/trump-rides-high-with...,5/2/2025,14:11:51 10 May 2025
3,Donald Trump (2nd term),All polls,TIPP Insights,NaN,4/30/2025,5/2/2025,1400.0,A,1.522330,1.172862,NaN,42.00,47.00,-5.000000,43.27937,48.78312,-5.503750,https://tippinsights.com/trump-rides-high-with...,5/2/2025,14:11:47 10 May 2025
4,Donald Trump (2nd term),Strong,RMG Research,Napolitan News Service,4/30/2025,5/8/2025,3000.0,RV,1.165249,1.024134,NaN,28.00,39.00,-11.000000,28.38242,41.92186,-13.539440,https://napolitannews.org/jobapproval,5/8/2025,14:11:52 10 May 2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1160,Donald Trump (2nd term),Adults,YouGov,Economist,3/9/2025,3/11/2025,1699.0,A,0.669784,0.000000,NaN,47.00,47.00,0.000000,46.62419,48.42194,-1.797749,https://d3nkl3psvxxpe9.cloudfront.net/document...,3/11/2025,14:11:51 10 May 2025
1161,Donald Trump (2nd term),Voters,Echelon Insights,NaN,2/10/2025,2/13/2025,1010.0,LV,1.048833,0.000000,NaN,52.00,46.00,6.000000,51.52624,46.02738,5.498859,https://echeloninsights.com/in-the-news/feb-20...,2/13/2025,14:11:49 10 May 2025
1162,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,2/28/2025,3/6/2025,1500.0,LV,0.212362,0.000000,True,50.00,48.00,2.000000,47.87342,48.88434,-1.010918,https://www.rasmussenreports.com/public_conten...,3/6/2025,14:11:49 10 May 2025
1163,Donald Trump (2nd term),Strong,Global Strategy Group/GBAO (Navigator Research),NaN,4/3/2025,4/7/2025,1000.0,RV,0.826071,0.000000,NaN,23.00,43.00,-20.000000,24.91729,43.19773,-18.280440,https://navigatorresearch.org/wp-content/uploa...,4/7/2025,14:11:52 10 May 2025


In [189]:
approval = approval[~approval['subgroup'].isin(['Weak', 'Strong', 'Economy', 'Cost', 'Immigration', 'Trade', 'Adults', 'Voters'])]

In [190]:
approval.to_csv('trump_approval_polls.csv')

In [191]:
approval.columns.values

array(['president', 'subgroup', 'pollster', 'sponsors', 'startdate',
       'enddate', 'samplesize', 'population', 'weight', 'influence',
       'tracking', 'approve', 'disapprove', 'net', 'adjusted_approve',
       'adjusted_disapprove', 'adjusted_net', 'url', 'createddate',
       'timestamp'], dtype=object)

In [192]:
approval.shape

(259, 20)

In [193]:
# dates = approval['Dates'].str.extract(r'(\d{1,2}\/\d{1,2}) \- (\d{1,2}\/\d{1,2}), (\d{4})')
# dates

In [194]:
# dates['end_date'] = dates[1] + '/' + dates[2]
# dates['end_date'] = pd.to_datetime(dates['end_date'])
# dates

In [195]:
# approval['Dates'] = dates['end_date']
# approval.head()

In [196]:
# samples = approval['Sample'].str.extract(r'[0-9,]{3,5} (\w{1,2})@@(\d{3,4})')
# samples

In [197]:
# approval['population'] = samples[0]
# approval['sample_size'] = samples[1].astype(int)
# approval.head()

In [198]:
ratings = pd.read_csv('pollster_ratings_silver.csv')
ratings

,Pollster,Grade,Predictive Plus-Minus,Mean-reverted bias,Number of polls,cat
0,AtlasIntel,A+@@1,-0.957740,D+0.16@@0.16334188,91,D+
1,Marquette University Law School,A/B@@4,-0.920261,D+0.32@@0.31545374,17,D+
2,The Washington Post,A+@@1,-0.902281,D+0.90@@0.89811021,98,D+
3,Research & Polling Inc.,A@@2,-0.871136,R+0.55@@-0.54944503,44,R+
4,The New York Times/Siena College,A@@2,-0.763947,D+1.41@@1.4121077,116,D+
...,...,...,...,...,...,...
530,Nielson Brothers Polling,C/D@@12,1.946864,D+3.05@@3.0515239,11,D+
531,Ascend Action,C/D@@12,2.282717,D+3.35@@3.3485439,6,D+
532,Jayhawk Consulting Services,C/D@@12,2.353968,D+2.27@@2.2674108,2,D+
533,Fort Hays State University,C/D@@12,2.489170,R+1.10@@-1.0960908,6,R+


In [199]:
ratings['Pollster'].values

array(['AtlasIntel', 'Marquette University Law School',
       'The Washington Post', 'Research & Polling Inc.',
       'The New York Times/Siena College', 'Cygnal', 'SurveyUSA',
       'Siena College', 'Beacon Research/Shaw & Co. Research',
       'Alaska Survey Research', 'OnMessage Inc.', 'TIPP Insights',
       'Emerson College', 'Landmark Communications',
       'Monmouth University', 'Data Orbital',
       'University of North Florida', 'MassINC Polling Group',
       'Marist College', 'Public Policy Institute of California',
       'Public Policy Polling', 'University of California Berkeley',
       'CBS News/The New York Times', 'Christopher Newport University',
       'Mason-Dixon Polling & Strategy', 'Strategic Vision LLC',
       'Suffolk University', 'Stockton University',
       'Field Research Corp. (Field Poll)', 'Quinnipiac University',
       'Los Angeles Times',
       'Hart Research Associates/Public Opinion Strategies',
       'Public Opinion Strategies', 'Susquehan

In [200]:
ratings[ratings['Pollster'].str.contains('North Star')]#.sum()

,Pollster,Grade,Predictive Plus-Minus,Mean-reverted bias,Number of polls,cat
75,North Star Opinion Research,B/C@@8,0.043339,R+0.63@@-0.62697512,13,R+
425,Univision/Latino Decisions/North Star Opinion ...,B/C@@8,0.830409,D+1.06@@1.0564657,7,D+


In [201]:
banned_pollsters = ratings[ratings['Grade'].str.contains('F')]['Pollster'].values
banned_pollsters

array(['Strategic Vision LLC', 'Pharos Research Group', 'Research 2000',
       'Big Data Poll', 'Overtime Politics', 'Rethink Priorities',
       'Blumenthal Research Daily', 'CSP Polling', 'KG Polling',
       'OurProgress (The Progress Campaign)', 'TCJ Research'],
      dtype=object)

In [202]:
approval = approval[~approval['pollster'].isin(banned_pollsters)]
approval.shape

(259, 20)

In [203]:
approval = pd.merge(left=approval, right=ratings[['Pollster', 'Predictive Plus-Minus']], left_on='pollster',
                   right_on='Pollster', how='inner')
approval

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,...,disapprove,net,adjusted_approve,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,Predictive Plus-Minus
0,Donald Trump (2nd term),All polls,Morning Consult,NaN,5/2/2025,5/8/2025,45000.0,A,1.236251,1.236251,...,48.04,-1.799999,45.31192,49.17940,-3.867481,https://pro.morningconsult.com/trackers/global...,5/8/2025,14:11:47 10 May 2025,Morning Consult,0.584994
1,Donald Trump (2nd term),All polls,TIPP Insights,NaN,4/30/2025,5/2/2025,1400.0,A,1.522330,1.172862,...,47.00,-5.000000,43.27937,48.78312,-5.503750,https://tippinsights.com/trump-rides-high-with...,5/2/2025,14:11:47 10 May 2025,TIPP Insights,-0.571372
2,Donald Trump (2nd term),All polls,RMG Research,Napolitan News Service,4/30/2025,5/8/2025,3000.0,RV,0.946469,0.888340,...,49.00,0.000000,45.96604,51.16201,-5.195969,https://napolitannews.org/jobapproval,5/8/2025,14:11:47 10 May 2025,RMG Research,0.278023
3,Donald Trump (2nd term),All polls,Emerson College,NaN,4/25/2025,4/28/2025,1000.0,RV,1.392550,0.758341,...,45.30,-0.599998,44.62696,47.82122,-3.194260,https://emersoncollegepolling.com/trump-100-days/,4/28/2025,14:11:47 10 May 2025,Emerson College,-0.569153
4,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,5/2/2025,5/8/2025,1500.0,LV,0.713778,0.713778,...,48.00,3.000000,48.57333,49.04616,-0.472832,https://www.rasmussenreports.com/public_conten...,5/8/2025,14:11:47 10 May 2025,Rasmussen Reports,0.087015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,Donald Trump (2nd term),All polls,McLaughlin & Associates,NaN,1/22/2025,1/27/2025,1000.0,LV,0.690477,0.000000,...,43.00,9.000000,50.64670,43.33120,7.315502,https://mclaughlinonline.com/pols/wp-content/u...,1/27/2025,14:11:47 10 May 2025,McLaughlin & Associates,1.412199
245,Donald Trump (2nd term),All polls,OnMessage Inc.,Rescuing the American Dream,2/17/2025,2/19/2025,1000.0,LV,1.404286,0.000000,...,49.00,1.000000,49.47305,48.60305,0.869999,https://onmessageinc.com/wp-content/uploads/20...,2/19/2025,14:11:47 10 May 2025,OnMessage Inc.,-0.600249
246,Donald Trump (2nd term),All polls,McLaughlin & Associates,America's New Majority Project,3/4/2025,3/7/2025,2000.0,RV,0.756492,0.000000,...,49.60,0.800003,49.04670,49.93120,-0.884499,https://americasnewmajorityproject.com/wp-cont...,3/7/2025,14:11:47 10 May 2025,McLaughlin & Associates,1.412199
247,Donald Trump (2nd term),All polls,RMG Research,Napolitan News Service,1/27/2025,1/31/2025,4000.0,RV,0.866070,0.000000,...,43.00,10.000000,49.96604,45.16201,4.804031,https://napolitannews.org/posts/president-trum...,1/31/2025,14:11:47 10 May 2025,RMG Research,0.278023


In [204]:
approval.shape

(249, 22)

In [205]:
approval['Pollster'].value_counts()

Pollster
Rasmussen Reports                                     76
YouGov                                                30
Morning Consult                                       19
RMG Research                                          16
Ipsos                                                 12
J.L. Partners                                          7
TIPP Insights                                          6
Global Strategy Group/GBAO (Navigator Research)        5
Emerson College                                        5
Echelon Insights                                       5
Quinnipiac University                                  4
SSRS                                                   4
Gallup                                                 4
AtlasIntel                                             4
McLaughlin & Associates                                4
Harris Insights & Analytics                            4
Clarity Campaign Labs                                  3
Cygnal                

In [206]:
# approval[~approval['tracking'].isna()]

In [207]:
approval['tracking'].value_counts()

tracking
True    79
Name: count, dtype: int64

In [208]:
approval = approval[approval['tracking'].isna() & (~approval['samplesize'].isna())]
approval

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,...,disapprove,net,adjusted_approve,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,Predictive Plus-Minus
0,Donald Trump (2nd term),All polls,Morning Consult,NaN,5/2/2025,5/8/2025,45000.0,A,1.236251,1.236251,...,48.04,-1.799999,45.31192,49.17940,-3.867481,https://pro.morningconsult.com/trackers/global...,5/8/2025,14:11:47 10 May 2025,Morning Consult,0.584994
1,Donald Trump (2nd term),All polls,TIPP Insights,NaN,4/30/2025,5/2/2025,1400.0,A,1.522330,1.172862,...,47.00,-5.000000,43.27937,48.78312,-5.503750,https://tippinsights.com/trump-rides-high-with...,5/2/2025,14:11:47 10 May 2025,TIPP Insights,-0.571372
2,Donald Trump (2nd term),All polls,RMG Research,Napolitan News Service,4/30/2025,5/8/2025,3000.0,RV,0.946469,0.888340,...,49.00,0.000000,45.96604,51.16201,-5.195969,https://napolitannews.org/jobapproval,5/8/2025,14:11:47 10 May 2025,RMG Research,0.278023
3,Donald Trump (2nd term),All polls,Emerson College,NaN,4/25/2025,4/28/2025,1000.0,RV,1.392550,0.758341,...,45.30,-0.599998,44.62696,47.82122,-3.194260,https://emersoncollegepolling.com/trump-100-days/,4/28/2025,14:11:47 10 May 2025,Emerson College,-0.569153
5,Donald Trump (2nd term),All polls,YouGov,Economist,5/2/2025,5/5/2025,1850.0,A,0.632122,0.591562,...,52.00,-10.000000,42.43221,52.68440,-10.252190,https://d3nkl3psvxxpe9.cloudfront.net/document...,5/5/2025,14:11:47 10 May 2025,YouGov,0.047635
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,Donald Trump (2nd term),All polls,McLaughlin & Associates,NaN,1/22/2025,1/27/2025,1000.0,LV,0.690477,0.000000,...,43.00,9.000000,50.64670,43.33120,7.315502,https://mclaughlinonline.com/pols/wp-content/u...,1/27/2025,14:11:47 10 May 2025,McLaughlin & Associates,1.412199
245,Donald Trump (2nd term),All polls,OnMessage Inc.,Rescuing the American Dream,2/17/2025,2/19/2025,1000.0,LV,1.404286,0.000000,...,49.00,1.000000,49.47305,48.60305,0.869999,https://onmessageinc.com/wp-content/uploads/20...,2/19/2025,14:11:47 10 May 2025,OnMessage Inc.,-0.600249
246,Donald Trump (2nd term),All polls,McLaughlin & Associates,America's New Majority Project,3/4/2025,3/7/2025,2000.0,RV,0.756492,0.000000,...,49.60,0.800003,49.04670,49.93120,-0.884499,https://americasnewmajorityproject.com/wp-cont...,3/7/2025,14:11:47 10 May 2025,McLaughlin & Associates,1.412199
247,Donald Trump (2nd term),All polls,RMG Research,Napolitan News Service,1/27/2025,1/31/2025,4000.0,RV,0.866070,0.000000,...,43.00,10.000000,49.96604,45.16201,4.804031,https://napolitannews.org/posts/president-trum...,1/31/2025,14:11:47 10 May 2025,RMG Research,0.278023


In [209]:
auxiliary = pd.read_csv('auxiliary_pollster_rating_stats.csv')
auxiliary

,Pollster,Correct calls,Outside MoE,House effect,ADPA,Herding penalty,herd_cat,p_cat
0,AtlasIntel,0.840659,8.8%@@91.208791,R+2.00@@-1.9990723,3.7@@3.6687982,0.096047,2,R+
1,The Washington Post,0.693878,6.1%@@93.877551,D+1.02@@1.0173736,3.6@@3.5756676,0.221841,3,D+
2,Research & Polling Inc.,0.886364,9.1%@@90.9090906,D+0.63@@0.63289553,4.6@@4.642076,0.286276,3,D+
3,The New York Times/Siena College,0.737069,18.1%@@81.896552,D+0.79@@0.79133403,3.2@@3.2483816,0.280332,3,D+
4,Cygnal,0.939655,13.8%@@86.206897,R+0.60@@-0.59704119,3.5@@3.4644768,0.521310,4,R+
...,...,...,...,...,...,...,...,...
93,Digital Research,0.904762,38.1%@@61.904761,R+2.44@@-2.4431372,11.4@@11.413808,0.000000,1,R+
94,Mitchell Research & Communications,0.695122,39%@@60.975611,R+0.45@@-0.44617203,4.4@@4.4410443,0.229193,3,R+
95,Merriman River Group,0.840000,60%@@39.999998,R+3.23@@-3.2347174,10.3@@10.260222,0.000000,1,R+
96,McLaughlin & Associates,0.689655,34.5%@@65.517241,R+4.94@@-4.9392128,6.2@@6.17804,0.000000,1,R+


In [210]:
house_eff = auxiliary['House effect'].str.extract(r'(\w)\+(\d\.\d\d)@@*.')
house_eff[1] = house_eff[1].astype(float)
house_eff

,0,1
0,R,2.00
1,D,1.02
2,D,0.63
3,D,0.79
4,R,0.60
...,...,...
93,R,2.44
94,R,0.45
95,R,3.23
96,R,4.94


In [211]:
# R --> -1, D --> 1
house_eff[0] = house_eff[0].replace({'R':-1, 'D':1})
house_eff['house_effect'] = house_eff[0] * house_eff[1]
house_eff

,0,1,house_effect
0,-1,2.00,-2.00
1,1,1.02,1.02
2,1,0.63,0.63
3,1,0.79,0.79
4,-1,0.60,-0.60
...,...,...,...
93,-1,2.44,-2.44
94,-1,0.45,-0.45
95,-1,3.23,-3.23
96,-1,4.94,-4.94


In [212]:
auxiliary['house_effect'] = house_eff['house_effect']
auxiliary = auxiliary.set_index(['Pollster'])
auxiliary

,Correct calls,Outside MoE,House effect,ADPA,Herding penalty,herd_cat,p_cat,house_effect
Pollster,,,,,,,,
AtlasIntel,0.840659,8.8%@@91.208791,R+2.00@@-1.9990723,3.7@@3.6687982,0.096047,2,R+,-2.00
The Washington Post,0.693878,6.1%@@93.877551,D+1.02@@1.0173736,3.6@@3.5756676,0.221841,3,D+,1.02
Research & Polling Inc.,0.886364,9.1%@@90.9090906,D+0.63@@0.63289553,4.6@@4.642076,0.286276,3,D+,0.63
The New York Times/Siena College,0.737069,18.1%@@81.896552,D+0.79@@0.79133403,3.2@@3.2483816,0.280332,3,D+,0.79
Cygnal,0.939655,13.8%@@86.206897,R+0.60@@-0.59704119,3.5@@3.4644768,0.521310,4,R+,-0.60
...,...,...,...,...,...,...,...,...
Digital Research,0.904762,38.1%@@61.904761,R+2.44@@-2.4431372,11.4@@11.413808,0.000000,1,R+,-2.44
Mitchell Research & Communications,0.695122,39%@@60.975611,R+0.45@@-0.44617203,4.4@@4.4410443,0.229193,3,R+,-0.45
Merriman River Group,0.840000,60%@@39.999998,R+3.23@@-3.2347174,10.3@@10.260222,0.000000,1,R+,-3.23


In [213]:
def apply_house_eff(data, pollster):
    df = data[data['pollster']==pollster]
    try:
        df['approve_adj'] = df['approve'] + (auxiliary.loc[pollster, 'house_effect'] / 2)
        df['disapprove_adj'] = df['disapprove'] - (auxiliary.loc[pollster, 'house_effect'] / 2)
    except KeyError:
        df['approve_adj'] = df['approve']
        df['disapprove_adj'] = df['disapprove']
    return df

In [214]:
pollsters = approval['pollster'].value_counts().index.values
df_adjs = []
for p in pollsters:
    df = apply_house_eff(approval, p)
    df_adjs.append(df)

approval_adj = pd.concat(df_adjs, axis=0)
approval_adj

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,...,adjusted_approve,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,Predictive Plus-Minus,approve_adj,disapprove_adj
5,Donald Trump (2nd term),All polls,YouGov,Economist,5/2/2025,5/5/2025,1850.0,A,0.632122,0.591562,...,42.43221,52.68440,-10.252190,https://d3nkl3psvxxpe9.cloudfront.net/document...,5/5/2025,14:11:47 10 May 2025,YouGov,0.047635,42.250,51.750
16,Donald Trump (2nd term),All polls,YouGov,NaN,4/25/2025,5/2/2025,2220.0,A,0.502594,0.316416,...,42.43221,52.68440,-10.252190,https://d3nkl3psvxxpe9.cloudfront.net/document...,5/2/2025,14:11:47 10 May 2025,YouGov,0.047635,42.250,51.750
21,Donald Trump (2nd term),All polls,YouGov,Economist,4/25/2025,4/28/2025,1785.0,A,0.427268,0.216843,...,42.43221,52.68440,-10.252190,https://d3nkl3psvxxpe9.cloudfront.net/document...,4/28/2025,14:11:47 10 May 2025,YouGov,0.047635,42.250,51.750
22,Donald Trump (2nd term),All polls,YouGov,Yahoo News,4/25/2025,4/28/2025,1597.0,A,0.408665,0.207402,...,42.43221,53.68440,-11.252190,https://www.scribd.com/document/855435184/2025...,4/28/2025,14:11:47 10 May 2025,YouGov,0.047635,42.250,52.750
31,Donald Trump (2nd term),All polls,YouGov,CBS News,4/23/2025,4/25/2025,2365.0,A,0.467547,0.146100,...,45.43221,55.68440,-10.252190,https://www.cbsnews.com/news/cbs-news-poll-tru...,4/25/2025,14:11:47 10 May 2025,YouGov,0.047635,45.250,54.750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,Donald Trump (2nd term),All polls,Targoz Market Research,Overton Insights,3/24/2025,3/28/2025,1200.0,RV,0.951969,0.000000,...,46.14761,50.68900,-4.541389,https://overtoninsights.com/wp-content/uploads...,3/28/2025,14:11:47 10 May 2025,Targoz Market Research,0.731655,46.570,50.430
105,Donald Trump (2nd term),All polls,Research Co.,NaN,1/22/2025,1/24/2025,1001.0,A,0.951747,0.000000,...,50.10303,45.37990,4.723129,https://researchco.ca/2025/01/27/immigration-usa/,1/24/2025,14:11:47 10 May 2025,Research Co.,0.632258,50.295,45.705
54,Donald Trump (2nd term),All polls,University of Massachusetts (Amherst),WCVB-TV,4/4/2025,4/9/2025,1000.0,A,0.983206,0.031716,...,44.27327,51.06240,-6.789131,https://www.umass.edu/political-science/about/...,4/9/2025,14:11:47 10 May 2025,University of Massachusetts (Amherst),0.536200,44.000,51.000
17,Donald Trump (2nd term),All polls,OH Predictive Insights,Center Square,4/15/2025,4/18/2025,2527.0,RV,1.628991,0.305279,...,44.08659,52.86376,-8.777172,https://bloximages.newyork1.vip.townnews.com/t...,4/18/2025,14:11:47 10 May 2025,OH Predictive Insights,-0.225885,45.065,51.935


In [215]:
approval.shape

(170, 22)

In [216]:
approval = approval_adj.copy()
approval.shape

(170, 24)

In [217]:
approval['poll_id'] = pd.factorize(approval['pollster'] + approval['enddate'])[0]
approval.head(7)

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,...,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,Predictive Plus-Minus,approve_adj,disapprove_adj,poll_id
5,Donald Trump (2nd term),All polls,YouGov,Economist,5/2/2025,5/5/2025,1850.0,A,0.632122,0.591562,...,52.6844,-10.25219,https://d3nkl3psvxxpe9.cloudfront.net/document...,5/5/2025,14:11:47 10 May 2025,YouGov,0.047635,42.25,51.75,0
16,Donald Trump (2nd term),All polls,YouGov,NaN,4/25/2025,5/2/2025,2220.0,A,0.502594,0.316416,...,52.6844,-10.25219,https://d3nkl3psvxxpe9.cloudfront.net/document...,5/2/2025,14:11:47 10 May 2025,YouGov,0.047635,42.25,51.75,1
21,Donald Trump (2nd term),All polls,YouGov,Economist,4/25/2025,4/28/2025,1785.0,A,0.427268,0.216843,...,52.6844,-10.25219,https://d3nkl3psvxxpe9.cloudfront.net/document...,4/28/2025,14:11:47 10 May 2025,YouGov,0.047635,42.25,51.75,2
22,Donald Trump (2nd term),All polls,YouGov,Yahoo News,4/25/2025,4/28/2025,1597.0,A,0.408665,0.207402,...,53.6844,-11.25219,https://www.scribd.com/document/855435184/2025...,4/28/2025,14:11:47 10 May 2025,YouGov,0.047635,42.25,52.75,2
31,Donald Trump (2nd term),All polls,YouGov,CBS News,4/23/2025,4/25/2025,2365.0,A,0.467547,0.146100,...,55.6844,-10.25219,https://www.cbsnews.com/news/cbs-news-poll-tru...,4/25/2025,14:11:47 10 May 2025,YouGov,0.047635,45.25,54.75,3
35,Donald Trump (2nd term),All polls,YouGov,The Times of London,4/21/2025,4/23/2025,1296.0,A,0.368719,0.095987,...,53.6844,-11.25219,https://d3nkl3psvxxpe9.cloudfront.net/document...,4/23/2025,14:11:47 10 May 2025,YouGov,0.047635,42.25,52.75,4
37,Donald Trump (2nd term),All polls,YouGov,Economist,4/19/2025,4/22/2025,1625.0,A,0.409293,0.095525,...,54.6844,-13.25219,https://d3nkl3psvxxpe9.cloudfront.net/document...,4/22/2025,14:11:47 10 May 2025,YouGov,0.047635,41.25,53.75,5


In [218]:
approval['population'] = approval['population'].str.strip(' ')

In [219]:
def pipeline(data: pd.DataFrame):
    dup = data[data.duplicated(subset='poll_id', keep=False)]
    ndup = data[~data.duplicated(subset='poll_id', keep=False)]
    # print('dup', dup, '\n ndup', ndup)
    dup_a = dup[dup['population'].isin(['A'])]
    # print('dup_a', dup_a)
    df = pd.concat([dup_a, ndup], axis=0)
    
    dup = df[df.duplicated(subset='poll_id', keep=False)]
    ndup = df[~df.duplicated(subset='poll_id', keep=False)]
    dup_lv = dup[dup['population'].isin(['RV'])]
    # print('dup_rv', dup_lv, 'ndup', ndup)
    df = pd.concat([dup_lv, ndup], axis=0)
    
    df['end_date'] = pd.to_datetime(df['end_date'])
    
    return df

In [220]:
approval = approval.rename({'enddate':'end_date', 'samplesize':'sample_size', 'Predictive Plus-Minus':'pred_plus_minus'}, axis=1)

In [221]:
approval['population'].value_counts().index.values

array(['RV', 'A', 'LV', 'V'], dtype=object)

In [222]:
approval.shape, pipeline(approval).shape

((170, 25), (162, 25))

In [223]:
pipeline(approval)

,president,subgroup,pollster,sponsors,startdate,end_date,sample_size,population,weight,influence,...,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,pred_plus_minus,approve_adj,disapprove_adj,poll_id
5,Donald Trump (2nd term),All polls,YouGov,Economist,5/2/2025,2025-05-05,1850.0,A,0.632122,0.591562,...,52.68440,-10.252190,https://d3nkl3psvxxpe9.cloudfront.net/document...,5/5/2025,14:11:47 10 May 2025,YouGov,0.047635,42.250,51.750,0
16,Donald Trump (2nd term),All polls,YouGov,NaN,4/25/2025,2025-05-02,2220.0,A,0.502594,0.316416,...,52.68440,-10.252190,https://d3nkl3psvxxpe9.cloudfront.net/document...,5/2/2025,14:11:47 10 May 2025,YouGov,0.047635,42.250,51.750,1
31,Donald Trump (2nd term),All polls,YouGov,CBS News,4/23/2025,2025-04-25,2365.0,A,0.467547,0.146100,...,55.68440,-10.252190,https://www.cbsnews.com/news/cbs-news-poll-tru...,4/25/2025,14:11:47 10 May 2025,YouGov,0.047635,45.250,54.750,3
35,Donald Trump (2nd term),All polls,YouGov,The Times of London,4/21/2025,2025-04-23,1296.0,A,0.368719,0.095987,...,53.68440,-11.252190,https://d3nkl3psvxxpe9.cloudfront.net/document...,4/23/2025,14:11:47 10 May 2025,YouGov,0.047635,42.250,52.750,4
37,Donald Trump (2nd term),All polls,YouGov,Economist,4/19/2025,2025-04-22,1625.0,A,0.409293,0.095525,...,54.68440,-13.252190,https://d3nkl3psvxxpe9.cloudfront.net/document...,4/22/2025,14:11:47 10 May 2025,YouGov,0.047635,41.250,53.750,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,Donald Trump (2nd term),All polls,Targoz Market Research,Overton Insights,3/24/2025,2025-03-28,1200.0,RV,0.951969,0.000000,...,50.68900,-4.541389,https://overtoninsights.com/wp-content/uploads...,3/28/2025,14:11:47 10 May 2025,Targoz Market Research,0.731655,46.570,50.430,161
105,Donald Trump (2nd term),All polls,Research Co.,NaN,1/22/2025,2025-01-24,1001.0,A,0.951747,0.000000,...,45.37990,4.723129,https://researchco.ca/2025/01/27/immigration-usa/,1/24/2025,14:11:47 10 May 2025,Research Co.,0.632258,50.295,45.705,162
54,Donald Trump (2nd term),All polls,University of Massachusetts (Amherst),WCVB-TV,4/4/2025,2025-04-09,1000.0,A,0.983206,0.031716,...,51.06240,-6.789131,https://www.umass.edu/political-science/about/...,4/9/2025,14:11:47 10 May 2025,University of Massachusetts (Amherst),0.536200,44.000,51.000,163
17,Donald Trump (2nd term),All polls,OH Predictive Insights,Center Square,4/15/2025,2025-04-18,2527.0,RV,1.628991,0.305279,...,52.86376,-8.777172,https://bloximages.newyork1.vip.townnews.com/t...,4/18/2025,14:11:47 10 May 2025,OH Predictive Insights,-0.225885,45.065,51.935,164


In [224]:
approval[approval['pollster']=='Harris Insights & Analytics']

,president,subgroup,pollster,sponsors,startdate,end_date,sample_size,population,weight,influence,...,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,pred_plus_minus,approve_adj,disapprove_adj,poll_id
47,Donald Trump (2nd term),All polls,Harris Insights & Analytics,Harvard CAPS,4/9/2025,4/10/2025,2286.0,RV,0.950531,0.054696,...,47.91776,-1.317207,https://harvardharrispoll.com/wp-content/uploa...,4/10/2025,14:11:47 10 May 2025,Harris Insights & Analytics,0.162447,48.105,45.895,118
101,Donald Trump (2nd term),All polls,Harris Insights & Analytics,Harvard CAPS,2/19/2025,2/20/2025,2443.0,RV,1.382092,0.000000,...,44.91776,5.682793,https://harvardharrispoll.com/wp-content/uploa...,2/20/2025,14:11:47 10 May 2025,Harris Insights & Analytics,0.162447,52.105,42.895,119
239,Donald Trump (2nd term),All polls,Harris Insights & Analytics,Harvard CAPS,3/26/2025,3/27/2025,2746.0,RV,1.124949,0.000000,...,47.91776,-0.317207,https://harvardharrispoll.com/key-results-marc...,3/27/2025,14:11:47 10 May 2025,Harris Insights & Analytics,0.162447,49.105,45.895,120


In [225]:
# banned_pollsters_aux = ['Rasmussen Reports', 'co/efficient', 'TIPP Insights', 'Civiqs', 'Trafalgar Group', 'OnMessage Inc.']
banned_pollsters_aux = pd.read_csv('banned_pollsters_aux.csv')['banned_pollsters'].values
banned_pollsters_aux

array(['Rasmussen Reports', 'co/efficient', 'TIPP Insights', 'Civiqs',
       'Trafalgar Group', 'OnMessage Inc.', 'McLaughlin & Associates',
       'North Star Opinion Research', 'Trafalgar Group/Insider Advantage'],
      dtype=object)

In [226]:
approval = approval[~approval['pollster'].isin(banned_pollsters_aux)]
approval.head()

,president,subgroup,pollster,sponsors,startdate,end_date,sample_size,population,weight,influence,...,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,pred_plus_minus,approve_adj,disapprove_adj,poll_id
5,Donald Trump (2nd term),All polls,YouGov,Economist,5/2/2025,5/5/2025,1850.0,A,0.632122,0.591562,...,52.6844,-10.25219,https://d3nkl3psvxxpe9.cloudfront.net/document...,5/5/2025,14:11:47 10 May 2025,YouGov,0.047635,42.25,51.75,0
16,Donald Trump (2nd term),All polls,YouGov,NaN,4/25/2025,5/2/2025,2220.0,A,0.502594,0.316416,...,52.6844,-10.25219,https://d3nkl3psvxxpe9.cloudfront.net/document...,5/2/2025,14:11:47 10 May 2025,YouGov,0.047635,42.25,51.75,1
21,Donald Trump (2nd term),All polls,YouGov,Economist,4/25/2025,4/28/2025,1785.0,A,0.427268,0.216843,...,52.6844,-10.25219,https://d3nkl3psvxxpe9.cloudfront.net/document...,4/28/2025,14:11:47 10 May 2025,YouGov,0.047635,42.25,51.75,2
22,Donald Trump (2nd term),All polls,YouGov,Yahoo News,4/25/2025,4/28/2025,1597.0,A,0.408665,0.207402,...,53.6844,-11.25219,https://www.scribd.com/document/855435184/2025...,4/28/2025,14:11:47 10 May 2025,YouGov,0.047635,42.25,52.75,2
31,Donald Trump (2nd term),All polls,YouGov,CBS News,4/23/2025,4/25/2025,2365.0,A,0.467547,0.146100,...,55.6844,-10.25219,https://www.cbsnews.com/news/cbs-news-poll-tru...,4/25/2025,14:11:47 10 May 2025,YouGov,0.047635,45.25,54.75,3


In [227]:
approval[approval['pollster']=='Harris Insights & Analytics']

,president,subgroup,pollster,sponsors,startdate,end_date,sample_size,population,weight,influence,...,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,pred_plus_minus,approve_adj,disapprove_adj,poll_id
47,Donald Trump (2nd term),All polls,Harris Insights & Analytics,Harvard CAPS,4/9/2025,4/10/2025,2286.0,RV,0.950531,0.054696,...,47.91776,-1.317207,https://harvardharrispoll.com/wp-content/uploa...,4/10/2025,14:11:47 10 May 2025,Harris Insights & Analytics,0.162447,48.105,45.895,118
101,Donald Trump (2nd term),All polls,Harris Insights & Analytics,Harvard CAPS,2/19/2025,2/20/2025,2443.0,RV,1.382092,0.000000,...,44.91776,5.682793,https://harvardharrispoll.com/wp-content/uploa...,2/20/2025,14:11:47 10 May 2025,Harris Insights & Analytics,0.162447,52.105,42.895,119
239,Donald Trump (2nd term),All polls,Harris Insights & Analytics,Harvard CAPS,3/26/2025,3/27/2025,2746.0,RV,1.124949,0.000000,...,47.91776,-0.317207,https://harvardharrispoll.com/key-results-marc...,3/27/2025,14:11:47 10 May 2025,Harris Insights & Analytics,0.162447,49.105,45.895,120


In [228]:
approval.columns.values

array(['president', 'subgroup', 'pollster', 'sponsors', 'startdate',
       'end_date', 'sample_size', 'population', 'weight', 'influence',
       'tracking', 'approve', 'disapprove', 'net', 'adjusted_approve',
       'adjusted_disapprove', 'adjusted_net', 'url', 'createddate',
       'timestamp', 'Pollster', 'pred_plus_minus', 'approve_adj',
       'disapprove_adj', 'poll_id'], dtype=object)

In [229]:
approval_polls = approval[['pollster', 'sponsors', 'startdate', 'end_date', 'sample_size', 'population', 'approve', 'disapprove', 'approve_adj', 'disapprove_adj', 'net', 'url', 'pred_plus_minus', 'poll_id']]
approval_polls

,pollster,sponsors,startdate,end_date,sample_size,population,approve,disapprove,approve_adj,disapprove_adj,net,url,pred_plus_minus,poll_id
5,YouGov,Economist,5/2/2025,5/5/2025,1850.0,A,42.0,52.0,42.250,51.750,-10.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,0
16,YouGov,NaN,4/25/2025,5/2/2025,2220.0,A,42.0,52.0,42.250,51.750,-10.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,1
21,YouGov,Economist,4/25/2025,4/28/2025,1785.0,A,42.0,52.0,42.250,51.750,-10.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,2
22,YouGov,Yahoo News,4/25/2025,4/28/2025,1597.0,A,42.0,53.0,42.250,52.750,-11.0,https://www.scribd.com/document/855435184/2025...,0.047635,2
31,YouGov,CBS News,4/23/2025,4/25/2025,2365.0,A,45.0,55.0,45.250,54.750,-10.0,https://www.cbsnews.com/news/cbs-news-poll-tru...,0.047635,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,NewsNation,NaN,4/23/2025,4/27/2025,1448.0,RV,44.0,56.0,44.000,56.000,-12.0,https://www.newsnationnow.com/politics/recessi...,0.614459,160
113,Targoz Market Research,Overton Insights,3/24/2025,3/28/2025,1200.0,RV,46.0,51.0,46.570,50.430,-5.0,https://overtoninsights.com/wp-content/uploads...,0.731655,161
105,Research Co.,NaN,1/22/2025,1/24/2025,1001.0,A,50.0,46.0,50.295,45.705,4.0,https://researchco.ca/2025/01/27/immigration-usa/,0.632258,162
54,University of Massachusetts (Amherst),WCVB-TV,4/4/2025,4/9/2025,1000.0,A,44.0,51.0,44.000,51.000,-7.0,https://www.umass.edu/political-science/about/...,0.536200,163


In [230]:
def avgs_pipeline(data: pd.DataFrame, date: datetime.date):
    state_pivot = pipeline(data)
    # state_pivot['end_date'] = pd.to_datetime(state_pivot['end_date'])
    # state_pivot['pred_plus_minus'] = state_pivot['pred_plus_minus'].map(lambda x: -x)
    state_pivot = state_pivot[state_pivot['end_date'] <= pd.to_datetime(date)]
    state_pivot['population'] = state_pivot['population'].astype(str).map(lambda x: x.strip())
    
    total_num_polls = state_pivot.shape[0]

    # Sample size weights
    total_sample_size = np.sum(state_pivot['sample_size'])
    state_pivot['sample_size_weights'] = (state_pivot['sample_size'].map(lambda x: np.sqrt(min(x, 3000))) / np.sqrt(np.median(state_pivot['sample_size'].map(lambda x: min(x, 2000)))))
    # print(np.median(state_pivot['sample_size'].map(lambda x: min(x, 5000))))
    state_pivot['sample_size_weights'] /= np.sum(state_pivot['sample_size_weights'])
    
    # Time weights
    # Variation of the equation used here: https://polls.votehub.us/
    latest_date = pd.Timestamp(date)
    delta = state_pivot['end_date'].apply(lambda x: (latest_date - x).days)# (latest_date - state_pivot['end_date']).apply(lambda x: x.days)
    linear_weights = (1 - delta/((latest_date - state_pivot['end_date'].min()).days + 1))
    exp_weights = 0.1**(delta/(40))
    state_pivot['time_weights'] =  0 * linear_weights + 1 * exp_weights
    # state_pivot['time_weights'] /= np.sum(state_pivot['time_weights'])
    
    # Quality weights
    rel_quality = state_pivot['pred_plus_minus']
    def quality_weight(rel_qual):
        if rel_qual > 0.5: # Utilizing predictive plus-minus from Silver Bulletin's ratings
            return 0.2
        # return np.sqrt(-rel_qual+1)+0.05
        return np.exp(-rel_qual/1.3)
    state_pivot['quality_weights'] = rel_quality.map(quality_weight)
    # state_pivot['quality_weights'] /= np.sum(state_pivot['quality_weights'])
    
    # Population weights
    def population_weight(population):
        if population == 'RV':
            return 0.8
        elif population == 'LV':
            return 0.6
        return 1
    state_pivot['population_weights'] = state_pivot['population'].map(population_weight)
    # state_pivot['population_weights'] /= np.sum(state_pivot['population_weights'])

    # Gather the weights together
    state_pivot['total_weights'] = state_pivot['sample_size_weights'] * state_pivot['time_weights'] * state_pivot['quality_weights'] * state_pivot['population_weights']
    state_pivot['total_weights'] /= np.sum(state_pivot['total_weights']) # Normalization step
    
    return state_pivot

In [231]:
approval_polls['sample_size'].isna().any()

False

In [232]:
avgs_pipeline(approval_polls, datetime.datetime.today())

,pollster,sponsors,startdate,end_date,sample_size,population,approve,disapprove,approve_adj,disapprove_adj,net,url,pred_plus_minus,poll_id,sample_size_weights,time_weights,quality_weights,population_weights,total_weights
5,YouGov,Economist,5/2/2025,2025-05-05,1850.0,A,42.0,52.0,42.250,51.750,-10.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,0,0.007313,0.749894,0.964021,1.0,0.055279
16,YouGov,NaN,4/25/2025,2025-05-02,2220.0,A,42.0,52.0,42.250,51.750,-10.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,1,0.008011,0.630957,0.964021,1.0,0.050951
31,YouGov,CBS News,4/23/2025,2025-04-25,2365.0,A,45.0,55.0,45.250,54.750,-10.0,https://www.cbsnews.com/news/cbs-news-poll-tru...,0.047635,3,0.008269,0.421697,0.964021,1.0,0.035147
35,YouGov,The Times of London,4/21/2025,2025-04-23,1296.0,A,42.0,53.0,42.250,52.750,-11.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,4,0.006121,0.375837,0.964021,1.0,0.023189
37,YouGov,Economist,4/19/2025,2025-04-22,1625.0,A,41.0,54.0,41.250,53.750,-13.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,5,0.006854,0.354813,0.964021,1.0,0.024513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,NewsNation,NaN,4/23/2025,2025-04-27,1448.0,RV,44.0,56.0,44.000,56.000,-12.0,https://www.newsnationnow.com/politics/recessi...,0.614459,160,0.006470,0.473151,0.200000,0.8,0.005121
113,Targoz Market Research,Overton Insights,3/24/2025,2025-03-28,1200.0,RV,46.0,51.0,46.570,50.430,-5.0,https://overtoninsights.com/wp-content/uploads...,0.731655,161,0.005890,0.084140,0.200000,0.8,0.000829
105,Research Co.,NaN,1/22/2025,2025-01-24,1001.0,A,50.0,46.0,50.295,45.705,4.0,https://researchco.ca/2025/01/27/immigration-usa/,0.632258,162,0.005380,0.002239,0.200000,1.0,0.000025
54,University of Massachusetts (Amherst),WCVB-TV,4/4/2025,2025-04-09,1000.0,A,44.0,51.0,44.000,51.000,-7.0,https://www.umass.edu/political-science/about/...,0.536200,163,0.005377,0.167880,0.200000,1.0,0.001888


In [233]:
def approval_over_time(data: pd.DataFrame):
    date_range = pd.date_range(start=datetime.date(2025, 1, 23), end=datetime.datetime.today(), freq='d', inclusive='both')
    yes_averages = []
    no_averages = []
    yes_stdevs = []
    no_stdevs = []
    for date in date_range:
        pipelined_df = avgs_pipeline(data, date)
        yes_avg = np.sum(pipelined_df['approve_adj'] * pipelined_df['total_weights'])
        no_avg = np.sum(pipelined_df['disapprove_adj'] * pipelined_df['total_weights'])
        yes_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['approve_adj'] - yes_avg)**2))
        no_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['disapprove_adj'] - no_avg)**2))
        yes_averages.append(yes_avg)
        no_averages.append(no_avg)
        yes_stdevs.append(yes_std)
        no_stdevs.append(no_std)
    
    return pd.DataFrame({'Date':date_range, 'Approve':yes_averages, 'Disapprove':no_averages}), pd.DataFrame({'Date':date_range, 'Approve':yes_stdevs, 'Disapprove':no_stdevs}), avgs_pipeline(data, datetime.datetime.today())

In [234]:
approval_avg, approval_std, polls_df = approval_over_time(approval_polls)
approval_avg

,Date,Approve,Disapprove
0,2025-01-23,51.310566,45.653535
1,2025-01-24,51.237797,45.330152
2,2025-01-25,51.237797,45.330152
3,2025-01-26,50.599984,45.294367
4,2025-01-27,49.443088,45.451175
...,...,...,...
103,2025-05-06,44.536341,51.277327
104,2025-05-07,44.536341,51.277327
105,2025-05-08,44.827254,51.082360
106,2025-05-09,44.827254,51.082360


In [235]:
polls_df.shape

(145, 19)

In [236]:
polls_df.head(3)

,pollster,sponsors,startdate,end_date,sample_size,population,approve,disapprove,approve_adj,disapprove_adj,net,url,pred_plus_minus,poll_id,sample_size_weights,time_weights,quality_weights,population_weights,total_weights
5,YouGov,Economist,5/2/2025,2025-05-05,1850.0,A,42.0,52.0,42.25,51.75,-10.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,0,0.007313,0.749894,0.964021,1.0,0.055279
16,YouGov,NaN,4/25/2025,2025-05-02,2220.0,A,42.0,52.0,42.25,51.75,-10.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,1,0.008011,0.630957,0.964021,1.0,0.050951
31,YouGov,CBS News,4/23/2025,2025-04-25,2365.0,A,45.0,55.0,45.25,54.75,-10.0,https://www.cbsnews.com/news/cbs-news-poll-tru...,0.047635,3,0.008269,0.421697,0.964021,1.0,0.035147


In [237]:
polls_df['net_adj'] = polls_df['approve_adj'] - polls_df['disapprove_adj']

In [238]:
display_df = polls_df[['pollster', 'sponsors', 'startdate', 'end_date', 'sample_size', 'population', 'approve', 'disapprove', 'net',
                      'total_weights', 'url']]
display_df['sponsors'] = '^Sponsor: ' + display_df['sponsors'] + '^'
display_df['sponsors'] = display_df['sponsors'].fillna('')
display_df = display_df.sort_values(['end_date'], ascending=False)
display_df = display_df.rename({'pollster':'Pollster', 'approve':'Approve', 'disapprove':'Disapprove', 'net':'Net', 'total_weights':'Weight', 'startdate':'Start Date', 'end_date':'End Date', 'url':'URL'}, axis='columns')
display_df['Sample'] = display_df['sample_size'].astype(int).astype(str) + ' ' + display_df['population']
# display_df['end_date'] = display_df['end_date'].map(lambda x: datetime.datetime.strptime(datetime.datetime.fromtimestamp(x), '%m/%d/%Y').strftime('%Y-%m-%d'))
# display_df['Date'] = pd.to_datetime(display_df['startdate']).astype(str) + ' - ' + display_df['end_date'].astype(str)
display_df['Start Date'] = pd.to_datetime(display_df['Start Date']).astype(str)
display_df['End Date'] = display_df['End Date'].astype(str)
display_df = display_df.reset_index()
# display_df = display_df.drop(['sample_size', 'population'], axis='columns')
display_df['Pollster'] = '<a href="' + display_df['URL'] + '" style="color:black; text-decoration:underline;"><u>' + display_df['Pollster'] + '</u></a>' + display_df['sponsors']
# display_df['Pollster'] = '[' + display_df['Pollster'] + '](' + display_df['URL'] + ')'
display_df = display_df[['Pollster', 'Start Date', 'End Date', 'Sample', 'Weight', 'Approve', 'Disapprove', 'Net']]
# display_df = display_df.set_index(['Pollster'])
display_df

,Pollster,Start Date,End Date,Sample,Weight,Approve,Disapprove,Net
0,"<a href=""https://napolitannews.org/jobapproval...",2025-04-30,2025-05-08,3000 RV,0.056061,49.00,49.00,0.000000
1,"<a href=""https://pro.morningconsult.com/tracke...",2025-05-02,2025-05-08,45000 A,0.017357,46.24,48.04,-1.799999
2,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-05-02,2025-05-05,1850 A,0.055279,42.00,52.00,-10.000000
3,"<a href=""https://pro.morningconsult.com/tracke...",2025-05-02,2025-05-04,2263 RV,0.009580,46.00,52.00,-6.000000
4,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-04-25,2025-05-02,2220 A,0.050951,42.00,52.00,-10.000000
...,...,...,...,...,...,...,...,...
140,"<a href=""https://researchco.ca/2025/01/27/immi...",2025-01-22,2025-01-24,1001 A,0.000025,50.00,46.00,4.000000
141,"<a href=""https://x.com/ScottWRasmussen/status/...",2025-01-20,2025-01-23,3000 RV,0.000133,57.00,39.00,18.000000
142,"<a href=""https://cdn.atlasintel.org/ed3f5238-e...",2025-01-21,2025-01-23,1882 A,0.000341,50.10,49.70,0.399998
143,"<a href=""https://www.ipsos.com/en-us/reuters-i...",2025-01-20,2025-01-21,1077 A,0.000078,47.00,41.00,6.000000


In [239]:
display_df.to_csv('polls_display_table.csv')

In [240]:
approval_std

,Date,Approve,Disapprove
0,2025-01-23,3.702450,5.577774
1,2025-01-24,3.406119,5.195595
2,2025-01-25,3.406119,5.195595
3,2025-01-26,3.666745,4.786089
4,2025-01-27,3.694118,4.404223
...,...,...,...
103,2025-05-06,2.947951,3.165078
104,2025-05-07,2.947951,3.165078
105,2025-05-08,3.032574,3.128137
106,2025-05-09,3.032574,3.128137


In [241]:
approval_avg['Net'] = approval_avg['Approve'] - approval_avg['Disapprove']
approval_avg

,Date,Approve,Disapprove,Net
0,2025-01-23,51.310566,45.653535,5.657031
1,2025-01-24,51.237797,45.330152,5.907644
2,2025-01-25,51.237797,45.330152,5.907644
3,2025-01-26,50.599984,45.294367,5.305617
4,2025-01-27,49.443088,45.451175,3.991913
...,...,...,...,...
103,2025-05-06,44.536341,51.277327,-6.740986
104,2025-05-07,44.536341,51.277327,-6.740986
105,2025-05-08,44.827254,51.082360,-6.255106
106,2025-05-09,44.827254,51.082360,-6.255106


In [242]:
approval_trendline = pd.merge(left=approval_avg, right=approval_std, left_on='Date', right_on='Date', how='inner')
approval_trendline['approve_CI_upper'] = approval_trendline['Approve_x'] + 1.96*approval_trendline['Approve_y']
approval_trendline['approve_CI_lower'] = approval_trendline['Approve_x'] - 1.96*approval_trendline['Approve_y']
approval_trendline['disapprove_CI_upper'] = approval_trendline['Disapprove_x'] + 1.96*approval_trendline['Disapprove_y']
approval_trendline['disapprove_CI_lower'] = approval_trendline['Disapprove_x'] - 1.96*approval_trendline['Disapprove_y']
approval_trendline = approval_trendline.rename({
    'Approve_x':'approve_mean',
    'Disapprove_x':'disapprove_mean',
}, axis='columns').drop(['Approve_y', 'Disapprove_y', 'Net'], axis='columns')
approval_trendline

,Date,approve_mean,disapprove_mean,approve_CI_upper,approve_CI_lower,disapprove_CI_upper,disapprove_CI_lower
0,2025-01-23,51.310566,45.653535,58.567369,44.053763,56.585972,34.721099
1,2025-01-24,51.237797,45.330152,57.913791,44.561802,55.513519,35.146785
2,2025-01-25,51.237797,45.330152,57.913791,44.561802,55.513519,35.146785
3,2025-01-26,50.599984,45.294367,57.786804,43.413164,54.675102,35.913632
4,2025-01-27,49.443088,45.451175,56.683559,42.202617,54.083453,36.818897
...,...,...,...,...,...,...,...
103,2025-05-06,44.536341,51.277327,50.314324,38.758358,57.480879,45.073775
104,2025-05-07,44.536341,51.277327,50.314324,38.758358,57.480879,45.073775
105,2025-05-08,44.827254,51.082360,50.771099,38.883410,57.213509,44.951211
106,2025-05-09,44.827254,51.082360,50.771099,38.883410,57.213509,44.951211


In [243]:
# Export approval ratings
approval_trendline.to_csv('approval_stats.csv')

In [244]:
polls_df.shape

(145, 20)

In [245]:
line = px.line(approval_avg, x='Date', y=['Approve', 'Disapprove'])
# scatter = px.scatter(polls_df[polls_df['end_date'] >= pd.to_datetime('2025-01-24')], x='end_date', y=['approve', 'disapprove'])
line.update_traces(hovertemplate=None)
line.update_layout(hovermode='x unified')
# scatter.update_traces(hovertemplate=None, hoverinfo='skip')
fig_approve_CI = go.Figure([
    go.Scatter(
        name='Approve CI Upper Bound',
        x = approval_avg['Date'],
        y = approval_avg['Approve'] + 1.96*approval_std['Approve'],
        mode='lines',
        marker=dict(color='#8be8f2'),
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),
    go.Scatter(
        name='Approve CI Lower Bound',
        x = approval_avg['Date'],
        y = approval_avg['Approve'] - 1.96*approval_std['Approve'],
        mode='lines',
        marker=dict(color='#8be8f2'),
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),  
])
fig_disapprove_CI = go.Figure([
    go.Scatter(
        name='Disapprove CI Upper Bound',
        x = approval_avg['Date'],
        y = approval_avg['Disapprove'] + 1.96*approval_std['Disapprove'],
        mode='lines',
        marker=dict(color='#a3cbec'),
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),
    go.Scatter(
        name='Disapprove CI Lower Bound',
        x = approval_avg['Date'],
        y = approval_avg['Disapprove'] - 1.96*approval_std['Disapprove'],
        mode='lines',
        marker=dict(color='#a3cbec'),
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),  
])

fig = go.Figure(data=line.data + fig_approve_CI.data + fig_disapprove_CI.data)
fig.update_layout(
    title='President Trump Approval Rating',
    xaxis_title='Date',
    yaxis_title='%',
    legend_title='Legend',
    hovermode='x unified'
)
fig.show()

In [246]:
line = px.line(approval_avg, x='Date', y=['Net'])
scatter = px.scatter(polls_df[polls_df['end_date'] >= pd.to_datetime('2025-01-24')], x='end_date', y=['net'])
fig = go.Figure(data=line.data + scatter.data)
fig.add_hline(y=0, line_dash='dash')
fig.show()

In [247]:
# scatter = px.scatter(polls_df[polls_df['end_date'] >= pd.to_datetime('2025-01-24')], x='end_date', y=['net', 'net_adj'])
# fig = go.Figure(data=scatter.data)
# fig.add_hline(y=0, line_dash='dash')
# fig.show()